In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report
import numpy as np

In [2]:
train_data_dir = 'images/training'
test_data_dir = 'images/testing'

#set the image size
img_width, img_height = 224, 224

#create data generators with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

#create data generators without data augmentation for testing
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=5,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=5,
    class_mode='binary')

Found 2392 images belonging to 2 classes.
Found 597 images belonging to 2 classes.


In [5]:
#try without base model
#try adding dropout
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

for layer in base_model.layers[-3:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

precision = Precision()
recall = Recall()

def f1_score(y_true, y_pred):
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

model.compile(loss='binary_crossentropy',
              #use adam optimizer
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
              metrics=[f1_score, 'accuracy', precision, recall, 'AUC'])

epochs = 3
history = model.fit(
    train_generator,
    steps_per_epoch=None,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=None)

model.save('page_flip_classifier_vgg16.h5')

Epoch 1/3
479/479 ━━━━━━━━━━━━━━━━━━━━ 293s 607ms/step - f1_score: 0.5800 - loss: 0.7100 - val_f1_score: 0.6052 - val_loss: 0.5405
Epoch 2/3
479/479 ━━━━━━━━━━━━━━━━━━━━ 300s 625ms/step - f1_score: 0.6424 - loss: 0.5513 - val_f1_score: 0.7113 - val_loss: 0.1406
Epoch 3/3
479/479 ━━━━━━━━━━━━━━━━━━━━ 292s 608ms/step - f1_score: 0.7359 - loss: 0.3608 - val_f1_score: 0.7702 - val_loss: 0.0881


In [6]:
y_pred = model.predict(test_generator)
y_pred = (y_pred > 0.5).astype(int)

y_true = test_generator.classes

report = classification_report(y_true, y_pred)
print(report)

120/120 ━━━━━━━━━━━━━━━━━━━━ 19s 161ms/step
              precision    recall  f1-score   support

           0       0.45      0.43      0.44       290
           1       0.48      0.50      0.49       307

    accuracy                           0.47       597
   macro avg       0.47      0.47      0.47       597
weighted avg       0.47      0.47      0.47       597

